# HW8 (20')

<font size='4'>

For this assignment, it is a combination of jupyter notebook assignment and python scripts.

For Q1, please upload your outputs including codes and graphics to your own GitHub repository. <br> You will need to disclose your GitHub repository below.

For Q2, please submit this jupyter notebook as an HTML or PDF file.

First of all, print your name (First and Last) below.

In [1]:
print('Suhas Das')

Suhas Das


## 0. Import relevant packages

In [12]:
!pip install statsmodels

import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as smf


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Q1. Convert your HW7 to python scripts. (10')

<font size='4'>

- Under your working directory, there should be a folder called `self_py_fun`.
- Create a new python file called `HW8Fun.py` and move previously defined functions `produce_trun_mean_cov()`, `plot_trunc_mean()`, and `plot_trunc_cov()` to that file. Make sure you import proper packages.
- Create another main file `HW8_main.py`.
- Import relevant packages, modules, and/or function.
- Copy the global variables and call your functions inside `HW8_main.py`.
- A major difference compared to HW7 is that you are asked to save those figures to your local working environment.
    - Create a new directory `K114` under your current working directory.
    - For mean functions, please save it as a `Mean.png` output using `plt.savefig()` function.
    - The changes should be made within `HW8Fun.py` rather than `HW8_main.py`.
    - For covariance matrices, please save them as `Covariance_Target.png`, `Covariance_Non-Target.png`, and `Covariance_All.png` outputs using the same function above.
    - To summarize, there should be **four** figures under `K114` folder.
- Upload your entire work to your GitHub repository via push button.

In [18]:
# Provide your GitHub repository link below in the Markdown chunk. Remember to make it public and make the link clickable.
# Do not include sensitive information in your GitHub repository.
#HW8fun.py
import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

def produce_trun_mean_cov(input_signal, input_type, E_val):
    length = input_signal.shape[1] // E_val
    tar = input_signal[input_type == 1]
    ntar = input_signal[input_type == -1]
    tar_mean = np.zeros((E_val, length))
    ntar_mean = np.zeros((E_val, length))
    tar_cov = np.zeros((E_val, length, length))
    ntar_cov = np.zeros((E_val, length, length))
    all_cov = np.zeros((E_val, length, length))
    for e in range(E_val):
        s = e * length
        e_ = (e + 1) * length
        tar_mean[e] = np.mean(tar[:, s:e_], axis=0)
        ntar_mean[e] = np.mean(ntar[:, s:e_], axis=0)
        tar_cov[e] = np.cov(tar[:, s:e_], rowvar=False)
        ntar_cov[e] = np.cov(ntar[:, s:e_], rowvar=False)
        all_cov[e] = np.cov(input_signal[:, s:e_], rowvar=False)
    return [tar_mean, ntar_mean, tar_cov, ntar_cov, all_cov]

def plot_trunc_mean(tar_mean, ntar_mean, time_index, labels, save_dir):
    plt.figure(figsize=(12, 8))
    for i in range(tar_mean.shape[0]):
        plt.plot(time_index, tar_mean[i], label=f'{labels[i]} (Target)')
        plt.plot(time_index, ntar_mean[i], label=f'{labels[i]} (Non-Target)', linestyle='--')
    plt.xlabel("Time (ms)")
    plt.ylabel("Amplitude")
    plt.title("Truncated Mean ERP Signals")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, "Mean.png"))
    plt.close()

def plot_trunc_cov(cov_list, save_dir):
    names = ["Target", "Non-Target", "All"]
    for i, cov in enumerate(cov_list):
        avg_cov = np.mean(cov, axis=0)
        plt.imshow(avg_cov, cmap='viridis', aspect='auto')
        plt.colorbar()
        plt.title(f"Covariance - {names[i]}")
        plt.savefig(os.path.join(save_dir, f"Covariance_{names[i]}.png"))
        plt.close()

bp_low = 0.5
bp_upp = 6
electrode_num = 16
electrode_name_ls = ['F3', 'Fz', 'F4', 'T7', 'C3', 'Cz', 'C4', 'T8', 'CP3', 'CPz', 'CP4', 'P7', 'P3', 'Pz', 'P4', 'P8']
parent_dir = '/Users/suhasdas/Documents/GitHub/BIOS-584'
data_dir = os.path.join(parent_dir, 'data')
subject = 'K114'
session = '001_BCI_TRN'
time_index = np.linspace(0, 800, 25)

out_dir = os.path.join(parent_dir, subject)
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

mat_path = os.path.join(data_dir, f"{subject}_{session}_Truncated_Data_0.5_6.mat")
data = sio.loadmat(mat_path)
signal = data['Signal']
labels = np.squeeze(data['Type'], axis=1)

results = produce_trun_mean_cov(signal, labels, electrode_num)
tar_mean, ntar_mean, tar_cov, ntar_cov, all_cov = results

plot_trunc_mean(tar_mean, ntar_mean, time_index, electrode_name_ls, out_dir)
plot_trunc_cov([tar_cov, ntar_cov, all_cov], out_dir)


## Q2. A real-world data anlaysis using `Pandas` and `Scipy` (10')

<font size='4'>

- Back to the `PTSD dataset.xlsx`, let's import the dataset and name it `ptsd_df`. (no point since everyone has done it a couple of times before.)

In [1]:
# Write your own code


### Q2.1. Univariate comparison (3')

<font size='4'> 
    
- Suppose that we would like to examine the utility/effect of an intervention program for patients with PTSD.
- We measure PCL5 scores at completion (`pcl5week_score.completion`) and PCL5 score at 3-month follow-up (`pcl5month_score.3_month_follow_up`). Let's assume the first score is pre-intervention and the second score is post-intervention.
- Report the summary statistics for each variable including mean, std, median, Q1, and Q3.
- Note that each patient will receive such two PCL5 scores. Use a appropriate statistical test to perform the univariate comparison. Report the outputing statistic and p-value.
- Before you run the statistic test, determine the data type and check the missingness of two columns. In particular, report the number of NA values for each variable.

In [2]:
# Write your own code


### Q2.2. Multiple Linear Regression (7')

<font size='4'>

- Select columns specified in the following code chunk and create a subset dataset named `ptsd_sub_df`.
- Fit a linear regression to examine the association between `caps_intake` (continuous outcome) and the remaining covariates (as predictors) using `ptsd_sub_df`.
    - Note that all covariates ending with `_code` are categorical variables.
- Use the instruction here to write the formula for linear regression in Python.
    - https://www.statsmodels.org/stable/example_formulas.html
- Report the output page including R2, adjusted R2, and parameter estimates, SE, 95% confidence intervals, and p-values.
- Provide a brief interpretation for all significant predictors (p<0.05) excluding the intercept.
- Relevant label information includes:
    - `employment_code`: 1: Employed, 2: Unemployed, 3: Retired, 4: Disabled/Unable to work, 5: Student, 6: Other.
    - `rank_code`: 1. Enlisted, 2: Officer, 3: Other

In [89]:
# The following column names are used for linear regression.
# Do not delete.
relevant_col_names = ['caps_intake', 'age_iop', 'gender_code', 'sexualorient_code', 'race_code', 'ethnicity_code', 
                      'education_code', 'employment_code',
                      'rank_code', 'branch_code', 'mdd_code', 'ctq_total_score', 'sexual_trauma', 'sud_code']

In [3]:
# Write your own code


In [88]:
# Write your interpretations below:
